In [1]:
from scipy.special import expit #to calculate logit 
import scipy.io as si
import numpy as np

#.mat file has two matrix X (5000*400) and y(5000*1)
File = si.loadmat('ex4data1.mat')

#sigmoid function implementation
def sigmoid_function(z):
          return expit(z)

#sigmoid gradient implementation
def sigmoid_gradient(Z):
    return (Z*(1 - Z))

#implement Forward Propagation
def compute_input_layer(theta1,input_layer):
    return (sigmoid_function(np.dot(input_layer,theta1.T)))

def compute_hidden_layer(theta2,A):
    return (sigmoid_function(np.dot(A,theta2.T)))

#return 5000*10 matrix
def compute_hypothesis(Theta1,Theta2,input_layer):
    
    #get 5000*25 matrix
    A1 = compute_input_layer(Theta1,input_layer)
    
    #change A dim to 5000*26
    A1 = np.insert(A1,0,1,axis = 1)

    #get 5000*10 matrix
    A2 = compute_hidden_layer(Theta2,A1)
    return A2 , A1

#implement Cost function for Neural Nertwork with no Regularization

#implement Cost function for Neural Nertwork with  Regularization
def Cost_function_reg(t_theta1,t_theta2,input_layer,actual_result,K,Lambda):
    
    #create theta1 and theta2
    theta1 = t_theta1
    theta1 = np.delete(theta1,0,1)
    theta2 = t_theta2
    theta2 = np.delete(theta2,0,1)
    
    #calculate hypothesis
    hypothesis1 , hypothesis2 = compute_hypothesis(t_theta1,t_theta2,input_layer)
    
    
    term1 =  - np.sum(np.multiply(actual_result,np.log(hypothesis1))) 
    term2 =  - np.sum(np.multiply((1 -actual_result),np.log(1 - hypothesis1))) 
    cost = term1 + term2
    cost = (cost/(len(hypothesis1)))
    
    #calculate Regularizarion Term
    
    #minimize each theta presents in all_theta dictionary

    #regularize theta1
    t1 = np.sum(theta1*theta1) 
    
    #regularize theta2
    t2 = np.sum(theta2*theta2)
    
    Reg_term = (Lambda/(2*(len(hypothesis1)))) * (t1 + t2)
   
    #update cost
    cost = cost + Reg_term
    return cost


In [2]:
#load input_layer
input_layer = np.array(File['X'])
input_layer = np.insert(input_layer,0,1,axis = 1)

#create actual result of training set in form of {0,1}
def result_binary(actual_result,num_labels):
    y = np.zeros((len(actual_result),num_labels))
    for i in range(len(actual_result)):
        temp = actual_result[i]
        y[i][temp - 1] = 1
    return y

output_layer = result_binary(File['y'],10)

#randomly initializes theata1(25,401) and theta2(10,26) for a given epsilon


epsilon = 0.12
init_theta1 = np.random.uniform(-epsilon,epsilon,(25,401))
init_theta2 = np.random.uniform(-epsilon,epsilon,(10,26))

In [3]:
#implement BackPropagation
def BackPropagation_Reg(init_theta1,init_theta2,input_layer,output_layer,Lambda,m):

    t1 = init_theta1
    t2 = init_theta2
   
    #calculate forward propagation
    hypothesis1 , hypothesis2 = compute_hypothesis(init_theta1,init_theta2,input_layer)
    
    #computes small delta
    
    #compute delta_3
    delta_3 = hypothesis1 - output_layer
    
    #compute delta_2
    term1 = delta_3.dot(init_theta2)
    delta_2 = term1*(hypothesis2*(1 - hypothesis2))
    delta_2 = np.delete(delta_2,0,1)
   
    #compute gradient for layer 1 with regularization
    grad_1 = np.dot(input_layer.T,delta_2).T
    grad_1 = grad_1/m
    
    t1 = np.delete(t1,0,1)
    t1 = np.insert(t1,0,0,1)
    
    grad_1 = grad_1 + (Lambda/m)*t1
    
    #compute gradient for layer 2
    grad_2 = ((delta_3).transpose()).dot(hypothesis2)
    grad_2 = grad_2/m
    
    t2 = np.delete(t2,0,1)
    t2 = np.insert(t2,0,0,1)
    
    grad_2 = grad_2 + (Lambda/m)*t2
    
    
    return (grad_1 , grad_2)


In [4]:
#perform  Gradient Checking on a small neural network

#make a small neural network

#define sizes
input_layer_size = 3 #including bias unit
hidden_layer_size = 5 #including bias unit
num_labels = 3
m = 5
def debugInitializeWeights(fan_out,fan_in):
    W = np.zeros((fan_out,fan_in + 1))
    W = (np.sin(np.arange(1,np.size(W) + 1)).reshape(W.shape))/10
    return W
#put some random values
#randomly  initializes test_theta's with a range of (-.12,.12)
epsilon = 0.12
test_theta1 = debugInitializeWeights(hidden_layer_size ,input_layer_size )
test_theta2 = debugInitializeWeights(num_labels,hidden_layer_size )

#randomly generate input layer (5*2)
test_input_layer = debugInitializeWeights(m,input_layer_size - 1)
test_input_layer = np.insert(test_input_layer,0,1,axis = 1)
print(test_input_layer)

#randomly generate  actual result 
test_result = 1 + np.remainder(np.arange(1,m+1),num_labels)
print(test_result)

#create test actualresult in binary form
test_actual_result = np.zeros((m,num_labels))
for i in range(m):
         test_actual_result[i,test_result[i] - 1] = 1

print(test_actual_result)

#make a long test_THETA VECTOR 

print(test_theta1.shape)
print(test_theta2.shape)

[[ 1.          0.0841471   0.09092974  0.014112  ]
 [ 1.         -0.07568025 -0.09589243 -0.02794155]
 [ 1.          0.06569866  0.09893582  0.04121185]
 [ 1.         -0.05440211 -0.09999902 -0.05365729]
 [ 1.          0.0420167   0.09906074  0.06502878]]
[2 3 1 2 3]
[[ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]
(5, 4)
(3, 6)


In [5]:
def gradient_checking(test_theta1,test_theta2,epsilon,num_labels,Lambda,test_input_layer,test_result):
    
    GRADIENT_CHECK = np.zeros((len(test_theta1.flatten()) + len(test_theta2.flatten()),1))
    test_theta_L = np.concatenate((test_theta1.flatten(),test_theta2.flatten()),0)
    test_theta_L = test_theta_L.reshape(len(test_theta1.flatten()) + len(test_theta2.flatten()),1)
    
    for i in range(len(test_theta_L)):
        inc = np.zeros((len(test_theta_L),1))
        inc[i] = inc[i] + epsilon
    
        theta_plus = test_theta_L + inc
        theta_minus = test_theta_L - inc
    
        #unrolling for theta_plus
        pass_theta1 = np.array(theta_plus[0:test_theta1.shape[0]*test_theta1.shape[1]]).reshape(test_theta1.shape)
        pass_theta2 = np.array(theta_plus[test_theta1.shape[0]*test_theta1.shape[1]:]).reshape(test_theta2.shape)
    
        cost_plus = Cost_function_reg(pass_theta1,pass_theta2,test_input_layer,test_result,num_labels,Lambda)
    
        #unrolling for theta_minus
        pass_theta1 = np.array(theta_minus[0:test_theta1.shape[0]*test_theta1.shape[1]]).reshape(test_theta1.shape)
        pass_theta2 = np.array(theta_minus[test_theta1.shape[0]*test_theta1.shape[1]:]).reshape(test_theta2.shape)
    
        cost_minus = Cost_function_reg(pass_theta1,pass_theta2,test_input_layer,test_result,num_labels,Lambda)
    
        GRADIENT_CHECK[i] = (cost_plus - cost_minus)/(2*epsilon)
        
    return GRADIENT_CHECK


In [7]:
Lambda = 653
#make calls to backpropagation and Cost_function for small neural network
TEST_GRADIENT_1 , TEST_GRADIENT_2 = BackPropagation_Reg(test_theta1,test_theta2,test_input_layer,test_actual_result,Lambda,m)

GRADIENT_CHECK = gradient_checking(test_theta1,test_theta2,.0001,num_labels,Lambda,test_input_layer,test_actual_result)
#COMPARE OUTPUT OF GRADIENT_CHECKING AND BACKPROPAGATION

GRADIENT_CHECK1 = np.array(GRADIENT_CHECK[0:test_theta1.shape[0]*test_theta1.shape[1]]).reshape(test_theta1.shape)
GRADIENT_CHECK2 = np.array(GRADIENT_CHECK[test_theta1.shape[0]*test_theta1.shape[1]:]).reshape(test_theta2.shape)

print("output from Backpropagtion for theta 1 \n")
print(TEST_GRADIENT_1)
print("\noutput from Gradient Checking for theta 1 \n")
print(GRADIENT_CHECK1)

print("\noutput from Backpropagtion for theta 2 \n")
print(TEST_GRADIENT_2)
print("\noutput from Gradient Checking for theta 2 \n")
print(GRADIENT_CHECK2)

output from Backpropagtion for theta 1 

[[  1.23162247e-02   1.18755982e+01   1.84328876e+00  -9.88373189e+00]
 [  3.92471369e-03  -3.64897631e+00   8.58046725e+00   1.29210688e+01]
 [ -8.08459407e-03  -7.10488439e+00  -1.30598939e+01  -7.00769717e+00]
 [ -1.26669105e-02   1.29371759e+01   8.49251369e+00  -3.76012648e+00]
 [ -5.59342547e-03  -9.80809348e+00   1.95715273e+00   1.19230017e+01]]

output from Gradient Checking for theta 1 

[[  1.23162246e-02   1.18755982e+01   1.84328876e+00  -9.88373189e+00]
 [  3.92471369e-03  -3.64897631e+00   8.58046725e+00   1.29210688e+01]
 [ -8.08459407e-03  -7.10488439e+00  -1.30598939e+01  -7.00769717e+00]
 [ -1.26669105e-02   1.29371759e+01   8.49251369e+00  -3.76012648e+00]
 [ -5.59342547e-03  -9.80809348e+00   1.95715273e+00   1.19230017e+01]]

output from Backpropagtion for theta 2 

[[  0.30934772  12.03649153   1.99006383  -9.72557201 -12.36593432
   -3.50193005]
 [  0.108133    12.97718207   5.43421847  -7.05018037 -13.00456382
   -6.9558